In [1]:
import spiceypy
# Allows us to convert to ephemeris time

In [4]:
import datetime
date_today = datetime.datetime.today()

date_today = date_today.strftime("%Y-%m-%dT00:00:00")

In [5]:
print(date_today)

2024-09-28T00:00:00


In [7]:
spiceypy.furnsh("../kernels/lsk/naif0012.tls.txt")
spiceypy.furnsh("../kernels/spk/de432s.bsp")

In [8]:
et_today_midnight = spiceypy.utc2et(date_today)
print(et_today_midnight)

780753669.1823555


In [11]:
# spkgeo calculates the geometric state (position and velocity) of a target object (the earth) relative to an observer (being the sun)
earth_state_wrt_sun, earth_sun_light_time = spiceypy.spkgeo(targ=399, et=et_today_midnight, 
                                                            ref="ECLIPJ2000", obs=10)
print(earth_state_wrt_sun)

[ 1.49333121e+08  1.31194714e+07 -1.94466328e+03 -3.08353468e+00
  2.95698126e+01 -1.03326126e-03]


In [12]:
import math
earth_sun_distance = math.sqrt(earth_state_wrt_sun[0] ** 2.0
                               + earth_state_wrt_sun[1] ** 2.0
                               + earth_state_wrt_sun[2] ** 2.0)
print(earth_sun_distance)
# Roughly 150,000,000 (million) km

149908310.75173977


In [13]:
# Converting value into astronomical units
earth_sun_distance_au = spiceypy.convrt(earth_sun_distance, "km", "au")
print(earth_sun_distance_au)

1.0020751641502477


In [17]:
print(f'State vector of the Earth w.r.t. the Sun for {date_today} (midnight):\n'
      + f'{earth_state_wrt_sun}')

State vector of the Earth w.r.t. the Sun for 2024-09-28T00:00:00 (midnight):
[ 1.49333121e+08  1.31194714e+07 -1.94466328e+03 -3.08353468e+00
  2.95698126e+01 -1.03326126e-03]


In [20]:
import numpy as np

# Vector of the earth in respect to the sun converted to numpy array for easier manipulation
earth_state_wrt_sun = np.array(earth_state_wrt_sun)

# [:3] slices the first 3 coordinates of the array which represent the x, y and z coordinates
# np.linalg.norm computes the Euclidean norm (also known as magnitude) of the coordinates. Which provides the straight line distance of the earth from the sun
# This is the position
earth_sun_distance = np.linalg.norm(earth_state_wrt_sun[:3])

# This is the earth's orbital speed relative to the sun. Slices the array to get the last three elements, converted to the Eucilidean norm
earth_orb_speed_wrt_sun = np.linalg.norm(earth_state_wrt_sun[3:])

print(earth_sun_distance)
print(earth_orb_speed_wrt_sun)

149908310.75173977
29.73015306860409


In [ ]:
# Loads specific kernel file 
spiceypy.furnsh("../kernels/pck/gm_de431.bsp")
_, GM_SUN = spiceypy.bodvcd(bodyid=10, item="GM", maxn=1)


v_orb_func = lambda gm, r: np.sqrt(gm/r)